# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [18]:
# Data manipulation
import pandas as pd
import numpy as np
# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 90
# Display all cell outputs
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important;}</style>"))
#display(HTML("<style>.rendered_html { font-size: 12px; }</style>"))
CSS = """
.output {
    flex-direction: row;
}
div.output pre {
    font-size: 12pt;
}
"""
#font-size: 30px;
HTML('<style>{}</style>'.format(CSS))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2
# Visualizations
from matplotlib import pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

import requests
import json
import os
import csv

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [19]:
url_base = "http://sapvm2.hec.ca:8010/ERPsim/OData/erpsim.xsodata/"
json_format = '?$format=json'
mdp = "ERPSIM1"

In [20]:
liste_df = []
#path_dump = os.getcwd() + '/dump/'

dirpath = os.getcwd() + "/sets/"
df_entity = pd.read_csv(dirpath + "entitySets.csv")

for j in range(len(df_entity)):
    url_1 = url_base + df_entity['keys'].loc[j] + json_format
    response = requests.get(url_1,auth=("416K",mdp))
    data = response.json()
    df_temp = pd.DataFrame.from_dict(data['d']['results'])
    liste_df.append(df_temp)

## Préparation du DATAFRAME pour les ventes : df_sales

In [21]:
index_sales = df_entity[df_entity['keys'] == 'Sales'].index[0]
df_sales = liste_df[index_sales]

In [22]:
df_sales['QUANTITY'] = df_sales['QUANTITY'].astype(int)
df_sales['NET_VALUE'] = df_sales['NET_VALUE'].astype(float)
df_sales['NET_PRICE'] = df_sales['NET_PRICE'].astype(float)
df_sales['COST'] = df_sales['COST'].astype(float)

In [23]:
df_sales['unit_cost'] = df_sales['COST'] / df_sales['QUANTITY']
#df_sales[['SIM_ROUND','SIM_PERIOD','MATERIAL_DESCRIPTION','COST','QUANTITY','NET_PRICE','unit_cost']]

## Préparation du DATAFRAME pour le marché : df_market

In [24]:
index_market = df_entity[df_entity['keys'] == 'Market'].index[0]
df_market = liste_df[index_market]

In [25]:
df_market['AVERAGE_PRICE'] = df_market['AVERAGE_PRICE'].astype(float)
df_market['NET_VALUE'] = df_market['NET_VALUE'].astype(float)
df_market['QUANTITY'] = df_market['QUANTITY'].astype(int)

In [36]:
df_market_company = df_market[df_market['SALES_ORGANIZATION'] == 'Company'].drop(['ID','SALES_ORGANIZATION','__metadata'],axis=1)
df_market_market = df_market[df_market['SALES_ORGANIZATION'] == 'Market'].drop(['ID','SALES_ORGANIZATION','__metadata'],axis=1)

df_market_share = df_market_company.merge(df_market_market, on=['AREA','DISTRIBUTION_CHANNEL','MATERIAL_DESCRIPTION','SIM_PERIOD','SIM_ROUND'])
df_market_share['share_value'] = df_market_share['NET_VALUE_x'] / df_market_share['NET_VALUE_y']
df_market_share['share_quantity'] = df_market_share['QUANTITY_x'] / df_market_share['QUANTITY_y']

## Préparation du DATAFRAME pour les finances : df_finance

In [27]:
index_finance = df_entity[df_entity['keys'] == 'FinancialsPostings'].index[0]
df_finance = liste_df[index_finance]

In [28]:
df_finance['AMOUNT_ABS'] = df_finance['AMOUNT_ABS'].astype(float)
df_finance['AMOUNT'] = df_finance['AMOUNT'].astype(float)
df_finance['AMOUNT_INV'] = df_finance['AMOUNT_INV'].astype(float)
df_finance['AMOUNT_BS'] = df_finance['AMOUNT_BS'].astype(float)
df_finance['AMOUNT_IS'] = df_finance['AMOUNT_IS'].astype(float)

In [29]:
df_finance[['FS_LEVEL_3','FS_LEVEL_2']].drop_duplicates().sort_values(['FS_LEVEL_3'])
#df_finance['FS_LEVEL_3'].unique()

,FS_LEVEL_3,FS_LEVEL_2
7,Current assets,Assets
5,Equity,Liabilities and Owners' Equity
16,Interest expenses,"Sales, General, and Administrative Expenses"
6,Liabilities,Liabilities and Owners' Equity
0,Long-term assets,Assets
17,Marketing expenses,"Sales, General, and Administrative Expenses"
26,SG&A,"Sales, General, and Administrative Expenses"
12,None,Operating Expenses
15,None,Inventory Change
28,None,Revenues


In [30]:
#df_finance[['SIM_ROUND','SIM_STEP','FS_LEVEL_2','DEBIT_CREDIT_INDICATOR','AMOUNT_ABS']]pi
pd.pivot_table(df_finance[['SIM_ROUND','SIM_STEP','FS_LEVEL_2','AMOUNT']][df_finance['FS_LEVEL_2'].isin(["Sales, General, and Administrative Expenses","Operating Expenses","Cost of Goods Sold"])],
               values='AMOUNT',index=['SIM_ROUND','SIM_STEP'], aggfunc=np.sum).fillna(0)

AMOUNT
SIM_ROUND SIM_STEP           
01        01             0.00
          02             0.00
          03             0.00
          04             0.00
          05        139973.10
          06         29520.00
          07         29520.00
          08         12320.00
          09         13938.54
          10        160444.52
          11         30330.04
          12         29520.00
          13         29520.00
          14         29520.00
          15        170188.72
          16         29520.00
          17         29520.00
          18         19680.00
          19         18480.00
          20        179904.34
02        01         17341.94
          02          7139.44
          03         20364.96
          04         29073.66
          05        240505.35
          06         16784.46
          07         20175.54
          08         14620.00
          09         29520.00
          10        308713.05
          11         29269.08
          12         29770.92
          13         26478.21
          14         28473.27
          15        305159.24
          16         29520.00
          17         28559.37
          18         30480.63
          19         27216.21
          20        294447.52
03        01         29520.00
          02         29460.96
          03         28782.00
          04         23811.57
          05        297893.90
          06         57370.10
          07         52258.63
          08         21431.58
          09         60968.02
          10        285183.38
          11         44359.28
          12         63637.08
          13         44903.94
          14             0.00
          15        268329.35
          16         22493.78
          17         20721.94
          18         22923.18
          19             0.00
          20        241166.10
04        01             0.00

# Tableau de bord pour les ventes 

In [31]:
@interact
def croise_dynamique(valeur=['NET_VALUE','QUANTITY','NET_PRICE','unit_cost'],
                    colonne1 = ['aucune','DISTRIBUTION_CHANNEL','MATERIAL_CODE','AREA', 'CITY','REGION','MATERIAL_DESCRIPTION'],
                    colonne2 = ['aucune','DISTRIBUTION_CHANNEL','MATERIAL_CODE','AREA', 'CITY','REGION','MATERIAL_DESCRIPTION']):
    
    type_graph = 'line'
    taille_graph = (15,6)
    nb_lignes_table = 30
    
    if valeur in ('NET_PRICE','unit_cost'):
        aggregate = np.average
    else:
        aggregate = np.sum
    if colonne1 == 'aucune' and colonne2 == 'aucune':
        pd.pivot_table(df_sales,values=valeur,index=['SIM_ROUND','SIM_STEP'], aggfunc=aggregate).fillna(0).plot(kind= type_graph,figsize=taille_graph)   
        plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.show()
        return pd.pivot_table(df_sales,values=valeur,index=['SIM_ROUND','SIM_STEP'], aggfunc=aggregate).fillna(0)[-1*nb_lignes_table:]
    else:
        if colonne2 == 'aucune':
            colonnes = [colonne1]
        else:
            colonnes = [colonne1,colonne2]
        pd.pivot_table(df_sales,values=valeur,index=['SIM_ROUND','SIM_STEP'], columns=colonnes, aggfunc=aggregate).fillna(0).plot(kind= type_graph,figsize=taille_graph)
        plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.show()
        return pd.pivot_table(df_sales,values=valeur,index=['SIM_ROUND','SIM_STEP'], columns=colonnes, aggfunc=aggregate).fillna(0)[-1*nb_lignes_table:]

interactive(children=(Dropdown(description='valeur', options=('NET_VALUE', 'QUANTITY', 'NET_PRICE', 'unit_cost…

In [35]:
@interact
def croise_market(valeur=['NET_VALUE','QUANTITY','AVERAGE_PRICE'],
                    colonne1 = ['aucune','DISTRIBUTION_CHANNEL','SALES_ORGANIZATION','AREA','MATERIAL_DESCRIPTION'],
                    colonne2 = ['aucune','DISTRIBUTION_CHANNEL','SALES_ORGANIZATION','AREA','MATERIAL_DESCRIPTION']):
    
    type_graph = 'bar'
    taille_graph = (15,6)
    
    if valeur == 'AVERAGE_PRICE':
        aggregate = np.average
    else:
        aggregate = np.sum
        
    #if colonne1 == 'SALES_ORGANIZATION' or colonne2 == 'SALES_ORGANIZATION':
    #    aggregate = np.p
        
    if colonne1 == 'aucune' and colonne2 == 'aucune':
        pd.pivot_table(df_market,values=valeur,index=['SIM_ROUND','SIM_PERIOD'], aggfunc=aggregate).fillna(0).plot(kind= type_graph,figsize=taille_graph)
        plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.show()
        return pd.pivot_table(df_market,values=valeur,index=['SIM_ROUND','SIM_PERIOD'], aggfunc=aggregate).fillna(0)
    else:
        if colonne2 == 'aucune':
            colonnes = [colonne1]
        else:
            colonnes = [colonne1,colonne2]
        pd.pivot_table(df_market,values=valeur,index=['SIM_ROUND','SIM_PERIOD'], columns=colonnes, aggfunc=aggregate).fillna(0).plot(kind= type_graph,figsize=taille_graph)
        plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.show()
        return pd.pivot_table(df_market,values=valeur,index=['SIM_ROUND','SIM_PERIOD'], columns=colonnes, aggfunc=aggregate).fillna(0)

interactive(children=(Dropdown(description='valeur', options=('NET_VALUE', 'QUANTITY', 'AVERAGE_PRICE'), value…

In [33]:
A = pd.pivot_table(df_sales[['SIM_ROUND','SIM_STEP','COST','QUANTITY']],
         values="COST",index=['SIM_ROUND','SIM_STEP'],aggfunc=np.sum).fillna(0)
B = pd.pivot_table(df_sales[['SIM_ROUND','SIM_STEP','COST','QUANTITY']],
         values="QUANTITY",index=['SIM_ROUND','SIM_STEP'],aggfunc=np.sum).fillna(0)
C = A.merge(B,on=['SIM_ROUND','SIM_STEP'])


## Parts de marchés

In [37]:
@interact
def croise_market_market(valeur=['share_value','share_quantity','NET_VALUE','QUANTITY','AVERAGE_PRICE'],
                    colonne1 = ['aucune','DISTRIBUTION_CHANNEL','AREA','MATERIAL_DESCRIPTION'],
                    colonne2 = ['aucune','DISTRIBUTION_CHANNEL','AREA','MATERIAL_DESCRIPTION']):
    
    type_graph = 'bar'
    taille_graph = (15,6)
    
    if valeur == 'AVERAGE_PRICE':
        aggregate = np.average
    else:
        aggregate = np.sum
        
    #if colonne1 == 'SALES_ORGANIZATION' or colonne2 == 'SALES_ORGANIZATION':
    #    aggregate = np.p
        
    if colonne1 == 'aucune' and colonne2 == 'aucune':
        pd.pivot_table(df_market_share,values=valeur,index=['SIM_ROUND','SIM_PERIOD'], aggfunc=aggregate).fillna(0).plot(kind= type_graph,figsize=taille_graph)
        plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.show()
        return pd.pivot_table(df_market_share,values=valeur,index=['SIM_ROUND','SIM_PERIOD'], aggfunc=aggregate).fillna(0)
    else:
        if colonne2 == 'aucune':
            colonnes = [colonne1]
        else:
            colonnes = [colonne1,colonne2]
        pd.pivot_table(df_market_share,values=valeur,index=['SIM_ROUND','SIM_PERIOD'], columns=colonnes, aggfunc=aggregate).fillna(0).plot(kind= type_graph,figsize=taille_graph)
        plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.show()
        return pd.pivot_table(df_market_share,values=valeur,index=['SIM_ROUND','SIM_PERIOD'], columns=colonnes, aggfunc=aggregate).fillna(0)

interactive(children=(Dropdown(description='valeur', options=('share_value', 'share_quantity', 'NET_VALUE', 'Q…